# Kafka Producer

In [2]:
# Libraries
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
import csv
from pymongo import MongoClient

In [3]:
# Function to obtain the latest date in database
def obtain_latest_date(collection):
    
    # Obtain all documents, show only the dates
    cursor = collection.find({}, {'date':1, '_id':0})
    
    latest_time = dt.date(2001,12,30) # Initialize latest time
    for document in cursor:
        date = dt.datetime.strptime(document['date'],'%d/%m/%y').date()
        if date > latest_time:
            latest_time = date
        
    return latest_time

In [4]:
# Function to parse csv to dictionary
def convert_csv_to_dict(file_path, int_list, float_list):
    reader = csv.DictReader(open(file_path,'r'))
    content = list(reader)
    
    # Iterate over each row
    for row in content:
        # Convert string to int
        for col in int_list:
            row[col] = int(row[col])
            
        # Convert string to float
        for col in float_list:
            # Separate precipitation flag from value
            if col == 'precipitation ':
                item = row[col].strip()
                flag = item[-1]
                value = float(item[:len(item)-1])
                row['precipitation_flag'] = flag
                row['precipitation_value'] = value
                
                # Remove key
                row.pop('precipitation ')
            else:
                row[col] = float(row[col])
                
        # Set producer
        row['producer'] = '1'
                
    return content

In [5]:
# Function to publish message from Kafka producer
# Code referenced from Tutorial Week 10
def publish_message(producer_instance, topic_name, data, key):
    try:
        key_bytes = bytes(key, encoding="utf-8")
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        print('Message published successfully. ' + data)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [6]:
# Function to connect to Kafka producer
# Code referenced from Tutorial Week 10
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [7]:
# Main Function
def main():
    
    # Separate columns into int list and float list
    cl_int_list = ['GHI_w/m2','air_temperature_celcius']
    cl_float_list = ['max_wind_speed','windspeed_knots','relative_humidity','precipitation ', 'latitude','longitude']
    file_path = 'climate_streaming.csv'
    
    # Read and process climate_streaming data
    lst = convert_csv_to_dict(file_path,cl_int_list,cl_float_list)
    
    # Initialize Kafka producer
    topic = "FIT3182_ASSIGNMENT_2"
    print("Publishing records...")
    producer1 = connect_kafka_producer()
    
    # Initialize MongoDB Client
    client = MongoClient()
    db = client.fit3182_assignment_db
    collection = db.data
    
    # Find the latest date and the new date
    latest_date = obtain_latest_date(collection)
    new_date = latest_date + dt.timedelta(days=1)
    
    while True:
        
        # Obtain random data
        data = lst[random.randrange(0,len(lst))]
        
        # Initialize dates
        data['date_of_sending'] = dt.datetime.now().strftime('%d/%m/%y %H:%M:%S')
        data['date_of_creation'] = new_date.strftime('%d/%m/%y')
        
        # Publish message from Kafka producer
        producer_key = '1'
        publish_message(producer1, topic, str(data), producer_key)
        
        # Increment new date
        new_date = new_date + dt.timedelta(days=1)
        
        # Send data every 10 seconds
        sleep(10)

In [ ]:
# Call main function
if __name__ == '__main__':
    main()

Publishing records...
Message published successfully. {'latitude': -36.341, 'longitude': 143.636, 'air_temperature_celcius': 16, 'relative_humidity': 47.5, 'windspeed_knots': 5.4, 'max_wind_speed': 12.0, 'GHI_w/m2': 140, 'precipitation_flag': 'I', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:03:54', 'date_of_creation': '01/01/42'}
Message published successfully. {'latitude': -37.627, 'longitude': 149.261, 'air_temperature_celcius': 26, 'relative_humidity': 53.7, 'windspeed_knots': 8.7, 'max_wind_speed': 15.0, 'GHI_w/m2': 216, 'precipitation_flag': 'I', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:04:05', 'date_of_creation': '02/01/42'}
Message published successfully. {'latitude': -36.2111, 'longitude': 141.505, 'air_temperature_celcius': 24, 'relative_humidity': 55.5, 'windspeed_knots': 7.9, 'max_wind_speed': 15.0, 'GHI_w/m2': 196, 'precipitation_flag': 'I', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '

Message published successfully. {'latitude': -38.495, 'longitude': 146.944, 'air_temperature_celcius': 8, 'relative_humidity': 44.6, 'windspeed_knots': 3.9, 'max_wind_speed': 8.0, 'GHI_w/m2': 72, 'precipitation_flag': 'G', 'precipitation_value': 0.39, 'producer': '1', 'date_of_sending': '17/06/22 13:07:55', 'date_of_creation': '25/01/42'}
Message published successfully. {'latitude': -37.602, 'longitude': 149.295, 'air_temperature_celcius': 17, 'relative_humidity': 50.8, 'windspeed_knots': 9.0, 'max_wind_speed': 15.9, 'GHI_w/m2': 145, 'precipitation_flag': 'G', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:08:05', 'date_of_creation': '26/01/42'}
Message published successfully. {'latitude': -37.462, 'longitude': 148.089, 'air_temperature_celcius': 8, 'relative_humidity': 40.7, 'windspeed_knots': 5.3, 'max_wind_speed': 12.0, 'GHI_w/m2': 74, 'precipitation_flag': 'G', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:08:15', 'date_o

Message published successfully. {'latitude': -36.0856, 'longitude': 144.233, 'air_temperature_celcius': 11, 'relative_humidity': 43.2, 'windspeed_knots': 5.9, 'max_wind_speed': 14.0, 'GHI_w/m2': 100, 'precipitation_flag': 'I', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:11:55', 'date_of_creation': '18/02/42'}
Message published successfully. {'latitude': -37.293, 'longitude': 141.245, 'air_temperature_celcius': 10, 'relative_humidity': 45.7, 'windspeed_knots': 3.7, 'max_wind_speed': 8.0, 'GHI_w/m2': 89, 'precipitation_flag': 'G', 'precipitation_value': 0.01, 'producer': '1', 'date_of_sending': '17/06/22 13:12:05', 'date_of_creation': '19/02/42'}
Message published successfully. {'latitude': -36.0005, 'longitude': 143.1847, 'air_temperature_celcius': 17, 'relative_humidity': 52.5, 'windspeed_knots': 7.7, 'max_wind_speed': 16.9, 'GHI_w/m2': 143, 'precipitation_flag': 'I', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:12:15', '

Message published successfully. {'latitude': -37.95, 'longitude': 142.366, 'air_temperature_celcius': 16, 'relative_humidity': 53.6, 'windspeed_knots': 8.1, 'max_wind_speed': 15.0, 'GHI_w/m2': 133, 'precipitation_flag': 'G', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:15:55', 'date_of_creation': '14/03/42'}
Message published successfully. {'latitude': -36.3328, 'longitude': 146.0355, 'air_temperature_celcius': 7, 'relative_humidity': 37.2, 'windspeed_knots': 6.2, 'max_wind_speed': 16.9, 'GHI_w/m2': 66, 'precipitation_flag': 'G', 'precipitation_value': 0.08, 'producer': '1', 'date_of_sending': '17/06/22 13:16:05', 'date_of_creation': '15/03/42'}
Message published successfully. {'latitude': -35.2881, 'longitude': 142.5679, 'air_temperature_celcius': 17, 'relative_humidity': 38.7, 'windspeed_knots': 16.8, 'max_wind_speed': 22.9, 'GHI_w/m2': 159, 'precipitation_flag': 'I', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:16:15', 

Message published successfully. {'latitude': -34.965, 'longitude': 142.031, 'air_temperature_celcius': 20, 'relative_humidity': 62.6, 'windspeed_knots': 10.1, 'max_wind_speed': 15.0, 'GHI_w/m2': 153, 'precipitation_flag': 'I', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:19:56', 'date_of_creation': '07/04/42'}
Message published successfully. {'latitude': -36.9085, 'longitude': 141.2821, 'air_temperature_celcius': 15, 'relative_humidity': 47.3, 'windspeed_knots': 8.6, 'max_wind_speed': 22.9, 'GHI_w/m2': 131, 'precipitation_flag': 'I', 'precipitation_value': 0.0, 'producer': '1', 'date_of_sending': '17/06/22 13:20:06', 'date_of_creation': '08/04/42'}
Message published successfully. {'latitude': -36.6859, 'longitude': 146.8907, 'air_temperature_celcius': 16, 'relative_humidity': 51.5, 'windspeed_knots': 8.7, 'max_wind_speed': 15.0, 'GHI_w/m2': 135, 'precipitation_flag': 'G', 'precipitation_value': 0.02, 'producer': '1', 'date_of_sending': '17/06/22 13:20:16